In [2]:
import requests
from lxml import html
from sklearn.metrics.pairwise import cosine_similarity
import os
from bs4 import BeautifulSoup as BS
from heapq import heappush, nlargest
import heapq
import csv
import numpy as np
from tqdm import tqdm
from functools import reduce
import pandas as pd
from time import time 
import string
import nltk
from nltk.corpus import stopwords
import tokenize
from nltk.tokenize import word_tokenize
import unicodedata
import math
import json
import _pickle as pickle
import ast
from nltk.stem import PorterStemmer
porter = PorterStemmer()

# Q1.1

In [ ]:
# Creating a list of movies inorder to get the web pages
movie_list = []
urls = ['https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html',
        'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html']
       
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        movie_list.append(link['href'])
        with open('movie_list.txt', 'a') as f:
            f.write(link['href'] + "\n")

# Q1.2

# Getting all the WebPages

In [ ]:
for movie in range(len(movie_list)):
    try:
        response = requests.get(movie_list[movie])
    except:
        time.sleep(1201)  # 20 mins of sleep in case of blocking by wikipedia
        response = requests.get(movie_list[movie])
    soup = BeautifulSoup(response.text, 'html.parser')
    name = "article-"+ str(movie) +".html"
    page = str(soup)
    time.sleep(1)
    with open(name, 'a') as f:
        f.write(page)

# Q1.3

# Parsing

In [ ]:
# Here at path we have all the html files
path = './0-30k/'

In [ ]:
if not os.path.exists('tsv_files'):
    os.makedirs('tsv_files')

In [ ]:
# We want just to parse the intro paragraphs and plot paragraphs
def intro_plot(soup,index=0):
    try:
        Node = soup.find_all('p')[index]
        lst = [Node.text]
        nextNode = Node
        while True:
            nextNode = nextNode.next_sibling

            try:
                tag_name = nextNode.name
            except AttributeError:
                tag_name = ""
            if tag_name == "p":
                lst.append(nextNode.text)
            else:
                break
    except:
        lst = None
    return lst

In [ ]:
# parsing the desire informations
def soup_parser(soup,info=None,next_tag='td',class_name=None):
    if next_tag:
        element = soup.find('th', text=info)
        if not element:
            element = 'NA'
        else:
            if info=='Release date':
                element = element.find_next(next_tag).text.split()[0]
            else:
                element = element.find_next(next_tag).text
    else:
        element = soup.find('h1').text
    return element



In [ ]:
# creating the all tsv files
columns = ['title', 'intro', 'plot', 'film_name', 'director', 'producer', 'writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget']
exceptions = []
start = time()
for subdir, dirs,files in os.walk(path):
    for file in files:
        try:
            soup = BS(open(path+file, encoding="utf8"), "html.parser")
            title = soup_parser(soup=soup,next_tag=None,class_name='firstHeading')
            intro = intro_plot(soup=soup)
            plot = intro_plot(soup=soup,index=len(intro))
            index = 0
            while intro == ['\n']:
                index+=1
                intro = intro_plot(soup=soup,index=index)
                plot = intro_plot(soup=soup,index=len(intro)+index)
            director = soup_parser(soup=soup, info='Directed by')
            producer = soup_parser(soup=soup, info='Produced by')
            writer = soup_parser(soup=soup, info='Written by')
            starred = soup_parser(soup=soup, info='Starring')
            music = soup_parser(soup=soup, info='Music by')
            date = soup_parser(soup=soup,info='Release date')
            duration = soup_parser(soup=soup, info='Running time')
            country = soup_parser(soup=soup, info='Country')
            language = soup_parser(soup=soup, info='Language')
            budget = soup_parser(soup=soup, info='Budget')
            
            all_info = [title,intro,plot,title,director,producer,writer,starred,music,date,duration,country,language,budget]
            with open('tsv_files/%s.tsv' %(file[:-5]), 'wt',encoding="utf-8") as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(columns)
                tsv_writer.writerow(all_info)

        except:
            print('Could not parse html file:', file)
            exceptions.append(file)
print("Execution time is:",time()-start)

# Q2

Q2.0

In [3]:
# Data preprocessing
def f_clean(i):
    s_word = stopwords.words('english')
    i = i.replace("\\n","")
    i = i.lower()
    i = i.translate(str.maketrans("","",string.punctuation))
    word = word_tokenize(i) 
    
    filt = [w for w in word if not w in s_word]
    ps = PorterStemmer()
    stemmed = []
    for w in filt:
        stemmed.append(ps.stem(w))
    
    punctuation = list(string.punctuation)
    punctuation.append("''")
    
    without_punt = [w for w in stemmed if not w in punctuation]

    return without_punt



# Q2.1

In [ ]:
# create list of the links in movies inorder to make a dataframe of films
tree1 = html.parse(r"movies1.html")
movies1 = html.tostring(tree1)
soup1 = BS(movies1)

tree2 = html.parse(r"movies2.html")
movies2 = html.tostring(tree2)
soup2 = BS(movies2)

tree3 = html.parse(r"movies3.html")
movies3 = html.tostring(tree3)
soup3 = BS(movies3)

links = []
for link in soup1.findAll('a'):
    links.append(link.get('href'))
for link in soup2.findAll('a'):
    links.append(link.get('href'))
for link in soup3.findAll('a'):
    links.append(link.get('href'))
len(links)

In [ ]:
# Creating a data frame of all films with their links
c = ['url','title','intro','plot','film_name','director','producer','writer','starring','music',
           'release_date','runtime','country','language','budget']
df_films = pd.DataFrame(columns = c)
for i in range(30000):
    with open("tsv_files/article-{}.tsv".format(i)) as f:
        reader = csv.reader(f, delimiter='\t')
        l = list(reader)[1]
        r = [links[i]] + l[0:]
        df_films = df_films.append(pd.DataFrame([r],columns = c),ignore_index = True)

In [ ]:
# Save it to a csv file
df_films.to_csv('film.csv')

In [4]:
df_film = pd.DataFrame(pd.read_csv('film.csv'))   # in the film.csv we have a data of all films
df_film.rename(columns = {'Unnamed: 0':'film_id'}, inplace = True) 

In [9]:
# create the vocabulary and set a new column in the dataframe with the len on the plot+intro
def create_voc(row,vocabulary):
    film_id = row['film_id']
    try:
        text = f_clean(row['intro']+row['plot'])
    except Exception as e:
        print(film_id,e)
        return 0
    for w in text:
        if w not in vocabulary:
            vocabulary[w] = {film_id:1}
        else:
            if film_id not in vocabulary[w]:
                vocabulary[w][film_id] = 1
            else:
                vocabulary[w][film_id] += 1
    return len(text)
    

In [ ]:
df_film['plot'] = df_film['plot'].astype(str)
v = {}
df_film['len_text'] = df_film.apply(create_voc,axis=1, vocabulary = v)

In [10]:
#we map the keys of the vocabulary in a number
def map_voc(voc):       
    i = 0 
    new_voc = {}
    for e in voc.keys():
        new_voc[e] = i
        i +=1
    return new_voc

In [ ]:
 mp = map_voc(v) #map

In [ ]:
# Store data (serialize)
json.dump(mp, open("map.txt",'w'))
# Store data (serialize)
json.dump(v, open("voc.txt",'w'))

In [7]:
# Load data (deserialize)
mp = json.load(open("map.txt"))
voca = json.load(open("voc.txt"))

In [5]:
# The first search engine
def query1():
    a = input("Enter a Query: ")
    a = f_clean(a)     #we need to clean the input so we have the match between the word
    l = []
    for i in a:
        try:
            l1 = list(voca[i].keys())
            l += [l1]
        except Exception as e:
            print("Not in any films:"+str(a))
            return
    e = list(reduce(set.intersection, [set(item) for item in l ]))  # we find the intersection from all the lis
    p = df_film[df_film['film_id'].isin(e)]  #we select only the film that are in the intesection
    p.reset_index(inplace=True)
    return p[['title','intro','url']].head(10)

In [8]:
query1()

Enter a Query: love


,title,intro,url
0,An Arcadian Maid,['An Arcadian Maid is a 1910 American silent f...,https://en.wikipedia.org/wiki/An_Arcadian_Maid
1,Pocahontas (1910 film),"[""Pocahontas is a 1910 American silent short d...",https://en.wikipedia.org/wiki/Pocahontas_(1910...
2,Ramona (1910 film),"[""Ramona is a 1910 American short drama film d...",https://en.wikipedia.org/wiki/Ramona_(1910_film)
3,What the Daisy Said,['What the Daisy Said is a one-reel film (abou...,https://en.wikipedia.org/wiki/What_the_Daisy_Said
4,Brown of Harvard (1911 film),['Brown of Harvard is a 1911 silent film based...,https://en.wikipedia.org/wiki/Brown_of_Harvard...
5,For Her Sake,"[""For Her Sake is a 1911 American silent short...",https://en.wikipedia.org/wiki/For_Her_Sake
6,Put Yourself in His Place,['Put Yourself in His Place is a 1912 American...,https://en.wikipedia.org/wiki/Put_Yourself_in_...
7,Atlantis (1913 film),['Atlantis is a 1913 Danish silent film direct...,https://en.wikipedia.org/wiki/Atlantis_(1913_f...
8,The Quakeress,['The Quakeress is a 1913 silent era short cos...,https://en.wikipedia.org/wiki/The_Quakeress
9,The Avenging Conscience,"['The Avenging Conscience: or ""Thou Shalt Not ...",https://en.wikipedia.org/wiki/The_Avenging_Con...


# Q2.2

In [11]:
# Now we define a function to calculates the tf idf
def invertx_voc(voc):
    new_voc = {}
    for k in voc.keys():
        repetition = len(voc[k])
        IDF = math.log(30000/repetition)
        for elem in voc[k].keys():
            val = voc[k][elem]
            length = list(df_film[df_film['film_id'] == int(elem)]['len_text'])[0]
            
            tf = val/length
            
            if k not in new_voc:
                new_voc[k] = {elem : tf*IDF}
            else:
                new_voc[k][elem] = tf*IDF
    return new_voc

In [ ]:
tfldf = invertx_voc(voca)
json.dump(tfldf, open("tfidf.txt",'w'))

In [12]:
tfldf = json.load(open("tfidf.txt"))

In [13]:
# Second search engine with cosine similarity
def query2():
    def cosSim(row):
        film_vector = []
        for elem in cleaned_query:
            if(elem in tfldf):
                if(str(row['film_id']) in tfldf[elem]):
                    film_vector.append(tfldf[elem][str(row['film_id'])])
                else:
                    film_vector.append(0)
            else:
                film_vector.append(0)
        query_vector_idf = tfidf_query(cleaned_query)
        cos_sim = cosine_similarity([film_vector], [query_vector_idf])[0][0]
        return cos_sim
    def tfidf_query(q):
        tfidf_q = []
        for elem in q:
            tfidf_q.append(1)
        return tfidf_q

    query = input("Enter a Query: ")
    tfldf = json.load(open("tfidf.txt"))
    cleaned_query = f_clean(query)
    query_vector = tfidf_query(cleaned_query)
    df_film['Similarity'] = df_film.apply(cosSim, axis = 1)
    ndf = df_film[['title', 'intro','url', 'Similarity']]
    result = ndf[ndf['Similarity'] > 0.7].sort_values('Similarity', ascending = False).head(10)
    return result  

In [14]:
query2()

Enter a Query: love


,title,intro,url,Similarity
25,An Arcadian Maid,['An Arcadian Maid is a 1910 American silent f...,https://en.wikipedia.org/wiki/An_Arcadian_Maid,1.0
23472,Halik sa Hangin,"[""Halik sa Hangin (lit.\u2009Kiss in the Wind)...",https://en.wikipedia.org/wiki/Halik_sa_Hangin,1.0
23477,The Love Affair (film),['The Love Affair is a 2015 Filipino drama fil...,https://en.wikipedia.org/wiki/The_Love_Affair_...,1.0
23479,Ex with Benefits,['Ex with Benefits is a 2015 Filipino romantic...,https://en.wikipedia.org/wiki/Ex_with_Benefits,1.0
23482,Etiquette for Mistresses,['Etiquette for Mistresses [3] is a 2015 Phili...,https://en.wikipedia.org/wiki/Etiquette_for_Mi...,1.0
23483,The Prenup,['The Prenup is a 2015 Filipino romantic comed...,https://en.wikipedia.org/wiki/The_Prenup,1.0
23484,Everyday I Love You (film),"['Everyday, I Love You is a 2015 Filipino roma...",https://en.wikipedia.org/wiki/Everyday_I_Love_...,1.0
23486,Love Is Blind (2016 film),['Love is Blind is a 2016 Philippine fantasy-r...,https://en.wikipedia.org/wiki/Love_is_Blind_(2...,1.0
23487,Padre de Familia (film),['Padre de Familia is a 2016 independent famil...,https://en.wikipedia.org/wiki/Padre_de_Familia...,1.0
23490,Love Me Tomorrow (film),['Love Me Tomorrow is a 2016 Filipino romantic...,https://en.wikipedia.org/wiki/Love_Me_Tomorrow...,1.0


# Q3

In [15]:
# Third search engine with the most similarity of desired runtime and the film's runtime
def query3():
    q_min = input("DESIRE RUNTIME(in mins): ")
    inp = input("Enter The Query: ")
    inp = f_clean(inp)     #we need to clean the input
    l = []
    for i in inp:
        try:
            l1 = list(voca[i].keys())
            l += [l1]
        except Exception as e:
            print("Not in any films:"+str(inp))
            return
    e = list(reduce(set.intersection, [set(item) for item in l ]))  # we find the intersection from all the lis
    p = df_film[df_film['film_id'].isin(e)]  #we select only the film that are in the intesection
    p.reset_index(inplace=True)
    p = p[['title','intro','url','runtime']]
    sim = []
    # here we calculate sort-scoring
    for i in range(len(p.runtime)):
        try:
            resrun = int(p.runtime[i].split('minutes')[0].strip())
            qrun   = int(q_min.strip())
            calc   = abs(resrun - qrun)
            sim.append(calc)
        except:
            # in some fields that we don't have any data for runtimes we replace it by a very large number
            sim.append(int('1000000')) 
    p['Differrence'] = sim   
    p.sort_values('Differrence',ascending = True,inplace = True)
    return p.head(10) 

In [16]:
query3()

DESIRE RUNTIME(in mins): 60
Enter The Query: love


,title,intro,url,runtime,Differrence
1274,The Case of the Black Parrot,['The Case of the Black Parrot is a 1941 Ameri...,https://en.wikipedia.org/wiki/The_Case_of_the_...,60 minutes,0
722,His Private Secretary,['His Private Secretary is a 1933 American Pre...,https://en.wikipedia.org/wiki/His_Private_Secr...,60 minutes,0
169,Woman's Place,"[""Woman's Place is a 1921 American romantic co...",https://en.wikipedia.org/wiki/Woman%27s_Place,60 minutes,0
720,Headline Shooter,['Headline Shooter is a 1933 American pre-Code...,https://en.wikipedia.org/wiki/Headline_Shooter,60 minutes[3],0
1118,Escape to Paradise,['Escape to Paradise is a 1939 American film d...,https://en.wikipedia.org/wiki/Escape_to_Paradise,60 minutes,0
158,Love Never Dies (1921 film),['Love Never Dies is a 1921 American silent dr...,https://en.wikipedia.org/wiki/Love_Never_Dies_...,60 minutes,0
1961,The Blonde Bandit,['The Blonde Bandit is a 1950 American crime f...,https://en.wikipedia.org/wiki/The_Blonde_Bandit,60 minutes,0
153,Heedless Moths,['Heedless Moths is a 1921 American silent mel...,https://en.wikipedia.org/wiki/Heedless_Moths,60 minutes,0
5766,All at Sea (1935 film),['All at Sea is a 1935 British comedy film dir...,https://en.wikipedia.org/wiki/All_at_Sea_(1935...,60 minutes,0
143,A Splendid Hazard,['A Splendid Hazard is a 1920 American silent ...,https://en.wikipedia.org/wiki/A_Splendid_Hazar...,60 minutes,0


In [19]:
# The total search engines
def search_engines():
    q = input("Choose which engine to search[1,2 or 3](ex: 1): \n")
    if q == '1':
        print('** THE SIMPLE QUERY **\n')
        return query1()
    if q == '2':
        print('** THE QUERY WITH HIGHEST SIMILARITY **\n')
        return query2()
    if q == '3':
        print('** THE QUERY WITH HIGHEST SIMILARITY With DESIRED RUNTIME **\n')
        print('** The lower is the Differrence, the highest is the similarity **\n')
        return query3()
    else:
        print('** PLEASE TRY AGAIN WITH CHOOSING A SEARCH ENGINE **')
        

In [20]:
search_engines()

Choose which engine to search[1,2 or 3](ex: 1): 
3
** THE QUERY WITH HIGHEST SIMILARITY With DESIRED RUNTIME **

** The lower is the Differrence, the highest is the similarity **

DESIRE RUNTIME(in mins): 90
Enter The Query: Disney movies


,title,intro,url,runtime,Differrence
1,Sullivan's Travels,"[""Sullivan's Travels is a 1941 American comedy...",https://en.wikipedia.org/wiki/Sullivan%27s_Tra...,90 minutes,0
22,Condorman,"[""Condorman is a 1981 American adventure comed...",https://en.wikipedia.org/wiki/Condorman,90 minutes,0
51,Annie (1999 film),['Annie is a 1999 American made-for-television...,https://en.wikipedia.org/wiki/Annie_(1999_film),90 minutes,0
27,Benji the Hunted,"[""Benji the Hunted is a 1987 American children...",https://en.wikipedia.org/wiki/Benji_the_Hunted,89 minutes,1
35,Plunder & Lightning,['Plunder & Lightning is an animated televisio...,https://en.wikipedia.org/wiki/Plunder_%26_Ligh...,91 minutes,1
18,The Castaway Cowboy,['The Castaway Cowboy is a 1974 American adven...,https://en.wikipedia.org/wiki/The_Castaway_Cowboy,91 minutes,1
78,Beverly Hills Chihuahua 3: Viva la Fiesta!,['Beverly Hills Chihuahua 3: Viva la Fiesta! i...,https://en.wikipedia.org/wiki/Beverly_Hills_Ch...,89 minutes,1
90,The Little Kidnappers (1990 film),['The Little Kidnappers is a 1990 Canadian/Ame...,https://en.wikipedia.org/wiki/The_Little_Kidna...,92 minutes,2
19,Herbie Rides Again,['Herbie Rides Again is a 1974 American comedy...,https://en.wikipedia.org/wiki/Herbie_Rides_Again,88 minutes,2
46,Tom and Huck,"[""Tom and Huck is a 1995 American adventure co...",https://en.wikipedia.org/wiki/Tom_and_Huck,92 minutes,2


# Q4

In [21]:
from itertools import combinations

sequence = 'dataminingsapienza'

count = set()
done = False
for i in range(len(sequence), 1, -1):
    sub = list(combinations(sequence, i))
    for k in range(len(sub)):
        if sub[k] == sub[k][::-1]:
            count.add(len(sub[k]))
            print(sub[k])
            done = True
            break
    if done: break

print(max(count))

('a', 'a', 'i', 'n', 'i', 'a', 'a')
7
